In [1]:
import random
random.seed(42)

In [2]:
import json
import math
import typing

import gym
import numpy as np
from stable_baselines3 import A2C

from arm import Arm, generate_random_action
# from production_line import Box

In [3]:
with open('configs/arm_config.json', 'r') as f:
    temp = f.readline()
    arm_config = json.loads(temp)

In [4]:
ARM_1_RANGE = arm_config['ARM_1_RANGE']
ARM_2_RANGE = arm_config['ARM_2_RANGE']
MAX_VARIABLE_ARM_LENGTH_UNIT = arm_config['MAX_VARIABLE_ARM_LENGTH_UNIT']
MAX_VARIABLE_ANGLE_UNIT = arm_config['MAX_VARIABLE_ANGLE_UNIT']
MIN_ARM_ZOOM_LENGTH = arm_config['MIN_ARM_ZOOM_LENGTH']
MIN_ANGLE_DEGREE = arm_config['MIN_ANGLE_DEGREE']
REWARD_DISCOUNT_FACTOR = arm_config['REWARD_DISCOUNT_FACTOR']
TERMINATION_CONDITION = arm_config['TERMINATION_CONDITION']

In [5]:
arm = Arm(
    arm_1_range=ARM_1_RANGE, arm_2_range=ARM_2_RANGE, 
    min_arm_zoom_length=MIN_ARM_ZOOM_LENGTH, min_angle_degree=MIN_ANGLE_DEGREE,
    max_variable_arm_length_unit=MAX_VARIABLE_ARM_LENGTH_UNIT, max_variable_angle_unit=MAX_VARIABLE_ANGLE_UNIT,
    reward_discount_factor=REWARD_DISCOUNT_FACTOR, termination_condition=TERMINATION_CONDITION,
)

In [1]:
env = arm

model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
obs = env.reset()
total_rewards = []
rewards = []
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    rewards.append(reward)
    if done == True:
        obs = env.reset()
        total_rewards.append(rewards)
        rewards = []
total_rewards.append(rewards)